# Ingesta de Datos desde APIs con Python

Importar datos desde una API nos permite tener información actualizada y en tiempo real.

Este enfoque ahorra tiempo y garantiza que la información que presentes sea precisa y relevante.

Una API es un puente que permite que diferentes aplicaciones se comuniquen entre sí.


## 1. Entendiendo la notación JSON

JSON, o Notación de Objetos en JavaScript, es ligero y fácil de leer tanto para máquinas como para humanos

La API devuelve los datos en JSON de una forma similar a como se muestra a continuación:

Este formato es similar a los diccionarios de Python


In [1]:
# Código en formato JSON
# {
#  "main": {
#   "temp": 293.15
#  },
#  "weather": [
#    {
#      "description": "clear sky"
#    }
#  ],
#  "name": "London"
#}

## 2. Creando archivo de configuración JSON para evitar que mi clave API se filtre

### 2.1. Crear el archivo config.json


In [2]:
# Se crea un archivo .json con el siguiente contenido
#{
#  "api_key": "abcd1234efgh5678ijkl" # Clave API de ejemplo
#}

### 2.2. Cargar clave desde Python

#### 2.2.1. ¿Qué hace la línea `with open(...) as f:` de la celda de abajo?

Es una forma segura y ordenada de abrir archivos. Su ventaja es que con este método, Python se encarga automáticamente de cerrar el archivo al final del bloque, incluso si hay errores.

`with open('config.json') as f:`

Es equivalente a

`f = open('config.json')`

`# ... hacer cosas ...`

`f.close()`

#### 2.2.1.1. Ventajas de usar with:

- Evita errores por archivos abiertos.
- Más limpio y profesional.
- Más seguro si trabajas con muchos archivos o datos sensibles.


#### 2.2.1.2. ¿Qué hace `pd.json_normalize()` de la celda de abajo?

Convierte un diccionario JSON anidado (como el que devuelve `response.json()`) en un DataFrame plano de pandas, separando las claves internas con puntos (`.`).


In [3]:
#import json
#
#with open('config.json') as f:
#    config = json.load(f) # Convierte todo el contenido del archivo JSON en un diccionario de Python 
#                          # (con claves/keys y valores/values)
#api_key = config['api_key'] # Extrae el valor asociado a la clave "api_key" dentro del diccionario config

In [4]:
# Importando información de una API
import pandas as pd
import requests
import json

# Realizar una solicitud GET
lat = 51.50 # Localización vertical. Es decir, Norte-Sur
lon = -0.12 # Localización horizontal. Es decir, Oriente-Occidente
#city = 'London'

with open('config.json') as f:
    config = json.load(f) # Convierte todo el contenido del archivo JSON en un diccionario de Python 
#                         # (con claves/keys y valores/values)

# Para obtener una API key es necesario crear una cuenta en OpenWeatherMap
# La API key es una contraseña personal, por eso traigo el dato desde un archivo aparte. 
# Así, al compartir este código, no se filtra mi contraseña

api_key = config['api_key'] # Extrae el valor asociado a la clave "api_key" dentro del diccionario config

url = f'https://api.openweathermap.org/data/2.5/weather?lat={lat}&lon={lon}&appid={api_key}&units=metric&lang=es'

response = requests.get(url)

# Usando condicionales para consultar una tabla y mostrar las primeras filas si al consulta fue exitosa.

if response.status_code == 200:
    weather_data = response.json() # Diccionario anidado
    df = pd.json_normalize(weather_data) # Convirtiendo el diccionario a un DataFrame
    print(df.head())
# Si hay error, mostramos un mensaje explicativo
else:
    print(f'Error en la solicitud: {response.status_code}')

# Siempre cerramos la conexión para liberar recursos

                                             weather      base  visibility  \
0  [{'id': 802, 'main': 'Clouds', 'description': ...  stations       10000   

           dt  timezone       id                 name  cod  coord.lon  \
0  1751398572      3600  2634341  City of Westminster  200      -0.12   

   coord.lat  ...  main.sea_level  main.grnd_level  wind.speed  wind.deg  \
0       51.5  ...            1014             1009        4.63        40   

   clouds.all  sys.type   sys.id  sys.country  sys.sunrise  sys.sunset  
0          25         2  2075535           GB   1751341655  1751401249  

[1 rows x 26 columns]


### 2.3. Accediendo a un dato desde diccionario y desde DataFrame


In [5]:
temperature_dict = weather_data['main']['temp'] # Accediendo a datos desde el diccionario
print(f'Temperatura actual: \n{temperature_dict} °C')

Temperatura actual: 
28.24 °C


In [6]:
temperature_df = df['main.temp'] # Accediendo a datos desde el Dataframe
print(f'Temperatura actual en °C: \n{temperature_df}')

Temperatura actual en °C: 
0    28.24
Name: main.temp, dtype: float64
